In [1]:
# import modules
import pickle
import sys
sys.path.append('..') 
# the data structure for blocksworld is defined in the parent directory
from reasoners.visualization import visualize

/data/adithya/anaconda3/envs/reasoners/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tot_inference import ProntoQAState
# load the log
with open("../../logs/prontoqa_DFS/12182023-002452/algo_output/27.pkl", 'rb') as f:
    dfs_result = pickle.load(f)

In [5]:
dfs_result.terminal_state

['Polly is a cat.',
 'Cats are felines.',
 'So Polly is a feline.',
 'Felines are carnivores.',
 'So Polly is a carnivore.',
 'Every carnivore is a mammal.',
 'So Polly is a mammal.',
 'Mammals are warm-blooded.',
 'So Polly is warm-blooded.',
 'The answer is true.']

In [2]:
# customize the information to show (optional)
from reasoners.visualization.tree_snapshot import NodeData, EdgeData
# from reasoners.algorithm.mcts import MCTSNode

def blocksworld_node_data_factory(n) -> NodeData:
    return NodeData({"state": "\n".join(n.state) if n.state else "Not expanded"})
def blocksworld_edge_data_factory(n) -> EdgeData:
    # print(f'self eval: { n.reward_details["self_eval"]}')
    return EdgeData({"intuition": n.reward_details["intuition"], "action": n.action, "self_eval": n.reward_details["self_eval"]})

In [4]:
# let's go!
visualize(bfs_result, node_data_factory=blocksworld_node_data_factory, edge_data_factory=blocksworld_edge_data_factory)

Visualizer URL: https://www.llm-reasoners.net/visualizer/822941f4-1304-40ec-8d6e-62e2bd928ded?accessKey=e86a2faf


# Visualize for multiple MCTS Results

In [9]:
from tot_inference import ProntoQAState
# load the log
for i in [6, 9, 10, 11, 13, 14, 15, 16, 17]:
    with open(f"../../logs/MCTS_{i}.pkl", 'rb') as f:
        dfs_result = pickle.load(f)
    print(i)
    visualize(dfs_result, node_data_factory=blocksworld_node_data_factory, edge_data_factory=blocksworld_edge_data_factory)

    

6
Visualizer URL: https://www.llm-reasoners.net/visualizer/e9240249-7c22-48de-983e-78490eda95e5?accessKey=8cf78454
9
Visualizer URL: https://www.llm-reasoners.net/visualizer/54fd353c-76e4-498e-8dd0-1fa03b8d0644?accessKey=5dd944bd
10
Visualizer URL: https://www.llm-reasoners.net/visualizer/0206b98e-98d9-4311-b7e5-cd2d6c69691b?accessKey=4a9a18c2
11
Visualizer URL: https://www.llm-reasoners.net/visualizer/f0ba684a-e357-442f-8e9e-d2dc9a7bd006?accessKey=bdef0f9f
13
Visualizer URL: https://www.llm-reasoners.net/visualizer/020c78b7-94d5-45a3-a189-27358af470f6?accessKey=17264082
14
Visualizer URL: https://www.llm-reasoners.net/visualizer/71b96f18-fbec-473c-8338-b7f2e28a092e?accessKey=86895753
15
Visualizer URL: https://www.llm-reasoners.net/visualizer/6fd2b04d-4558-4478-a4e6-3b1138bab3b0?accessKey=bd44fcea
16
Visualizer URL: https://www.llm-reasoners.net/visualizer/7a17db6d-aa0e-409d-9e56-45766efd7c84?accessKey=d9828c0c
17
Visualizer URL: https://www.llm-reasoners.net/visualizer/d180907e-ac0d-

In [4]:
from tot_inference import ProntoQAState
# load the log
for i in [1]:
    with open(f"../../logs/MCTS_{i}.pkl", 'rb') as f:
        dfs_result = pickle.load(f)
    print(i)
    visualize(dfs_result, node_data_factory=blocksworld_node_data_factory, edge_data_factory=blocksworld_edge_data_factory)

1
Visualizer URL: https://www.llm-reasoners.net/visualizer/fbd123e4-19ba-4494-9940-77ce92e35f0a?accessKey=41e7009b


In [8]:
from tot_inference import ProntoQAState
import torch


def bfs_pronto_extractor(algo_output):
        if torch.distributed.is_initialized():
            torch.distributed.barrier()
        # to make sure the plan is saved before evaluation in multi-process setting
        try:
            answer = "\n".join(algo_output.terminal_node.state[2::2])
            answer = answer.replace("So ", "")
            return answer

        except Exception as e:
            print("Error in output extraction,", e)
            return ""
    
def dfs_bw_extractor(algo_output):
    if torch.distributed.is_initialized():
        torch.distributed.barrier()
    # to make sure the plan is saved before evaluation in multi-process setting
    try:
        answer = "\n".join(algo_output.terminal_state[2::2])
        answer = answer.replace("So ", "")
        return answer

    except Exception as e:
        print("Error in output extraction,", e)
        return ""
        

# load the log
for i in [1]:
    with open(f"../../logs/MCTS_{i}.pkl", 'rb') as f:
        dfs_result = pickle.load(f)
        search_algo="beam"
        output_extractor = dfs_bw_extractor if search_algo == "dfs" else bfs_pronto_extractor
        answer_extractor= lambda x: x.test_example.answer
        output = output_extractor(dfs_result)
        # answer = self.answer_extractor(example)

        # print(f"answer extractor: { answer}")
        print(f"output extractor: {output}")

output extractor: Alex is a lepidopteran.
Alex is an insect.
Alex is an arthropod.
Alex is not bony.
Alex is six-legged.


In [9]:
list_ex= [1,2,3,4,5,6,7,8,9]
print(list_ex[2::2])

[3, 5, 7, 9]


In [10]:
chain_of_thought=['Alex is a butterfly.', 'Each butterfly is a lepidopteran.', 'Alex is a lepidopteran.', 'Lepidopterans are insects.', 'Alex is an insect.', 'Every insect is six-legged.', 'Alex is six-legged.']
answer_extractor=lambda x: "\n".join(x[2::2])

print(answer_extractor(chain_of_thought))

Alex is a lepidopteran.
Alex is an insect.
Alex is six-legged.
